In [3]:
import json
from collections import defaultdict
import matplotlib.pyplot as plt
import re
import sys
import subprocess  

In [2]:
!pip install matplotlib

     |████████████████████████████████| 7.3 MB 5.1 MB/s eta 0:00:01
     |████████████████████████████████| 61 kB 216 kB/s  eta 0:00:01
     |████████████████████████████████| 895 kB 19.6 MB/s eta 0:00:01


In [4]:
def make_doc_command(step, dire):

    com = r"""../src/main.py 
            --dataset blogs_2dom_cleaned 
            --clean_mem_every 5 
            --reset_output_dir 
            --classifier_dir ../pretrained_classifer/blogs_2dom_cleaned 
            --train_src_file ../data/blogs_2dom_cleaned/dev_drop_10.txt 
            --train_trg_file ../data/blogs_2dom_cleaned/dev_drop_10.docs 
            --dev_src_file ../data/blogs_2dom_cleaned/dev_drop_10.txt 
            --dev_trg_file ../data/blogs_2dom_cleaned/dev_drop_10.docs 
            --dev_trg_ref ../data/blogs_2dom_cleaned/dev_ref.txt 
            --src_vocab  ../data/blogs_2dom_cleaned/text.vocab 
            --trg_vocab  ../data/blogs_2dom_cleaned/doc.attr.vocab 
            --d_word_vec=128 
            --d_model=512 
            --log_every=100 
            --eval_every=3000 
            --ppl_thresh=10000 
            --eval_bleu 
            --batch_size 32 
            --valid_batch_size 128 
            --patience 5 
            --lr_dec 0.5 
            --lr 0.001 
            --dropout 0.3 
            --max_len 10000 
            --seed 0 
            --beam_size 1 
            --word_blank 0.2 
            --word_dropout 0.1 
            --word_shuffle 3 
            --cuda 
            --anneal_epoch 5 
            --temperature 0.01 
            --klw 0.03 
            --max_pool_k_size 1 
            --bt 
            --lm 
            --gumbel_softmax 
            --avg_len 
            --run_classifier_doc_evaluation 
            --step {} 
            --no_styles 2 
            --input_classifier_text ../{} 
            --input_doc_dict ../data/blogs_2dom_cleaned/doc.dict""".format(step,dire)
    return com

#print (proc.communicate())

In [5]:
def make_entropy_command(step, dire):

    com = r"""../src/main.py 
        --dataset blogs_2dom_cleaned 
        --clean_mem_every 5 
        --reset_output_dir
        --classifier_dir ../pretrained_classifer/blogs_2dom_cleaned 
        --train_src_file ../data/blogs_2dom_cleaned/dev_drop_10.txt 
        --train_trg_file ../data/blogs_2dom_cleaned/dev_drop_10.attr 
        --dev_src_file ../data/blogs_2dom_cleaned/dev_drop_10.txt 
        --dev_trg_file ../data/blogs_2dom_cleaned/dev_drop_10.attr 
        --dev_trg_ref ../data/blogs_2dom_cleaned/dev_ref.txt 
        --src_vocab  ../data/blogs_2dom_cleaned/text.vocab 
        --trg_vocab  ../data/blogs_2dom_cleaned/attr_disc.vocab 
        --d_word_vec=128 
        --d_model=512 
        --log_every=100 
        --eval_every=3000 
        --ppl_thresh=10000 
        --eval_bleu 
        --batch_size 32 
        --valid_batch_size 128 
        --patience 5 
        --lr_dec 0.5 
        --lr 0.001
        --dropout 0.3
        --max_len 10000
        --seed 0 
        --beam_size 1 
        --word_blank 0.2 
        --word_dropout 0.1 
        --word_shuffle 3 
        --cuda 
        --anneal_epoch 5 
        --temperature 0.01 
        --klw 0.03
        --max_pool_k_size 1
        --bt
        --lm 
        --gumbel_softmax
        --avg_len
        --run_classifier_avg_certainty
        --step {}
        --no_styles 2
        --input_classifier_text ../{}""".format(step,dire)
    return com

#print (proc.communicate())

In [6]:
def make_confidence_ratio_command(step, dire):

    com = r"""../src/main.py 
        --dataset blogs_2dom_cleaned 
        --clean_mem_every 5 
        --reset_output_dir
        --classifier_dir ../pretrained_classifer/blogs_2dom_cleaned 
        --train_src_file ../data/blogs_2dom_cleaned/dev_drop_10.txt 
        --train_trg_file ../data/blogs_2dom_cleaned/dev_drop_10.attr 
        --dev_src_file ../data/blogs_2dom_cleaned/dev_drop_10.txt 
        --dev_trg_file ../data/blogs_2dom_cleaned/dev_drop_10.attr 
        --dev_trg_ref ../data/blogs_2dom_cleaned/dev_ref.txt 
        --src_vocab  ../data/blogs_2dom_cleaned/text.vocab 
        --trg_vocab  ../data/blogs_2dom_cleaned/attr_disc.vocab 
        --d_word_vec=128 
        --d_model=512 
        --log_every=100 
        --eval_every=3000 
        --ppl_thresh=10000 
        --eval_bleu 
        --batch_size 32 
        --valid_batch_size 128 
        --patience 5 
        --lr_dec 0.5 
        --lr 0.001
        --dropout 0.3
        --max_len 10000
        --seed 0 
        --beam_size 1 
        --word_blank 0.2 
        --word_dropout 0.1 
        --word_shuffle 3 
        --cuda 
        --anneal_epoch 5 
        --temperature 0.01 
        --klw 0.03
        --max_pool_k_size 1
        --bt
        --lm 
        --gumbel_softmax
        --avg_len
        --run_classifier_count_certainty
        --step {}
        --no_styles 2
        --input_classifier_text ../{}""".format(step,dire)
    return com

#print (proc.communicate())

In [7]:
def make_gpt_command_dom(step, dire,dom,alph):

    com = r"""/home/NAME/PROJECT/Deep-Learning/GPT2-HarryPotter-Training/examples/run_lm_finetuning.py
        --output_dir=/home/NAME/PROJECT/Deep-Learning/GPT2-HarryPotter-Training/examples/output-dom{}
        --model_type=gpt2
        --model_name_or_path=gpt2-medium
        --train_data_file=/home/NAME/PROJECT/style-pooling/{}/dev.trans_{}a{}
        --do_eval
        --eval_data_file=/home/NAME/PROJECT/style-pooling/{}/dev.trans_{}a{}
        --block_size=200
        --per_gpu_train_batch_size=1""".format(dom,dire,step,alph,dire,step,alph)
    return com

#print (proc.communicate())

In [8]:
def make_gpt_command_original(step, dire):

    com = r"""/home/name/NAME/PROJECT/Deep-Learning/GPT2-HarryPotter-Training/examples/run_lm_finetuning.py
        --output_dir=/home/NAME/PROJECT/Deep-Learning/GPT2-HarryPotter-Training/examples/output-original/checkpoint-1
        --model_type=gpt2
        --model_name_or_path=gpt2-medium
        --train_data_file=/home/NAME/PROJECT/style-pooling/{}/dev.trans_{}
        --do_eval
        --eval_data_file=/home//NAME/PROJECT/style-pooling/{}/dev.trans_{}
        --block_size=200
        --per_gpu_train_batch_size=1""".format(dire,step,dire,step)
    return com

#print (proc.communicate())

In [9]:
def make_gpt_command_tuned(step, dire):

    com = r"""/home/NAME/PROJECT/Deep-Learning/GPT2-HarryPotter-Training/examples/run_lm_finetuning.py
        --output_dir=/home/NAME/PROJECT/Deep-Learning/GPT2-HarryPotter-Training/examples/output/checkpoint-290000
        --model_type=gpt2
        --model_name_or_path=gpt2-medium
        --train_data_file=/home/NAME/PROJECT/style-pooling/{}/dev.trans_{}
        --do_eval
        --eval_data_file=/home/NAME/PROJECT/style-pooling/{}/dev.trans_{}
        --block_size=200
        --per_gpu_train_batch_size=1""".format(dire,step,dire,step)
    return com

#print (proc.communicate())

In [10]:
def make_split_cmd(step, dire):

    com = r"""split -l 19374 /home/NAME/PROJECT/style-pooling/{}/dev.trans_{} /home/NAME/PROJECT/style-pooling/{}/dev.trans_{}""".format(dire,step,dire,step)
    return com

#print (proc.communicate())

In [11]:
def make_lexi_cmd(step, dire):

    com = r"""../scripts/lexical_div.py --input /home/NAME/PROJECT/style-pooling/{}/dev.trans_{}""".format(dire,step)
    return com

#print (proc.communicate())

# sum + onelm + min not distinguished

In [13]:


val_step =[]
classifier_acc = []
n_acc = []
bt_acc = []
total_loss=[]
neg_ELBO = []
KL_loss=[]

confidence_accuracy = []
confidence_ratio = []


cls_sent = []
maj_doc = []
sum_doc = []

log_paths=[
"place log path here"
]
for log_path in log_paths:   
    with open("../"+log_path+"/stdout") as f:
        val_step =[]
        classifier_acc = []
        n_acc = []
        bt_acc = []
        total_loss=[]
        neg_ELBO = []
        KL_loss=[]

        cls_sent = []
        maj_doc = []
        sum_doc = []
        entorpy = []
        diff_per = []

        gpt_original =[]
        gpt_dom0_0 =[]
        gpt_dom0_1 =[]
        gpt_dom1_0 =[]
        gpt_dom1_1 =[]
        gpt_tuned =[]

        lex_div = []
        unique_words = []

        confidence_accuracy = []
        confidence_ratio = []
        for i, line in enumerate(f):
            if ("classifier_acc=" in line):
                list_acc = re.findall("\d+\.\d+", line)
                #print(list_acc)
                classifier_acc.append(float(list_acc[0]))
            elif ("val_step" in line):
                list_arguments = re.findall(r"[-+]?\d*\.\d+|\d+", line)
                #print(list_arguments)
                if (int(list_arguments[0]) >33000):
                    continue
                elif (int(list_arguments[0]) < 24000):
                    continue
                elif (int(list_arguments[0])% 1500 != 0):
                    continue
                val_step.append(int(list_arguments[0]))
                total_loss.append(float(list_arguments[1]))
                neg_ELBO.append(float(list_arguments[2]))
                KL_loss.append(float(list_arguments[3]))
                n_acc.append(float(list_arguments[6])*100)
                bt_acc.append(float(list_arguments[7])*100)
                
                ###get acc nums
                com = make_doc_command(str(val_step[-1]),log_path)
                proc = subprocess.Popen(["/home/name/anaconda3/envs/py37/bin/python"]+(com.replace('\n','').split()), stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
                outp = (proc.communicate())
                list_arguments = re.findall(r"[-+]?\d*\.\d+|\d+", str(outp[0]))
                sum_doc.append(float(list_arguments[-3]))
                maj_doc.append(float(list_arguments[-2]))
                cls_sent.append(float(list_arguments[-1]))
                #print(outp)
                ## get entropy
                com = make_entropy_command(str(val_step[-1]),log_path)
                proc = subprocess.Popen(["/home/name/anaconda3/envs/py37/bin/python"]+(com.replace('\n','').split()), stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
                outp = (proc.communicate())
                list_arguments = re.findall(r"[-+]?\d*\.\d+|\d+", str(outp[0]))
                diff_per.append(float(list_arguments[-2]))
                entorpy.append(float(list_arguments[-1]))
                ## get confidence ratio
                com = make_confidence_ratio_command(str(val_step[-1]),log_path)
                proc = subprocess.Popen(["/home/name/anaconda3/envs/py37/bin/python"]+(com.replace('\n','').split()), stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
                outp = (proc.communicate())
                list_arguments = re.findall(r"[-+]?\d*\.\d+|\d+", str(outp[0]))
                confidence_accuracy.append(float(list_arguments[-2]))
                confidence_ratio.append(float(list_arguments[-1]))               
                ## get lex div
                com = make_lexi_cmd(str(val_step[-1]),log_path)
                proc = subprocess.Popen(["/home/name/anaconda3/envs/py37/bin/python"]+(com.replace('\n','').split()), stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
                outp = (proc.communicate())
                list_arguments = re.findall(r"[-+]?\d*\.\d+|\d+", str(outp[0]))
                unique_words.append(float(list_arguments[-2]))
                lex_div.append(float(list_arguments[-1]))
                ##gpt
                ##### get gpt-original
                com = make_gpt_command_original(str(val_step[-1]),log_path)
                proc = subprocess.Popen("export CUDA_VISIBLE_DEVICES=2;/home/name/anaconda3/envs/py37/bin/python " +" ".join(com.replace('\n','').split()), cwd='/home/name/PROJECT/Deep-Learning/GPT2-HarryPotter-Training/examples', stdout=subprocess.PIPE, stderr=subprocess.STDOUT, shell=True,  executable = '/bin/sh')
            
                outp = (proc.communicate())
                list_arguments = re.findall(r"[-+]?\d*\.\d+|\d+", str(outp[0]))
                gpt_original.append(float(list_arguments[-1]))
                #print(outp)
                #print(float(list_arguments[-1]))


  
                #print(list_arguments)
        print("=SPLIT(\""+','.join([str(i) for i in val_step])+"\",\",\")")
        print("=SPLIT(\""+','.join([str(i) for i in bt_acc])+"\",\",\")")
        print("=SPLIT(\""+','.join([str(i*100) for i in sum_doc])+"\",\",\")")
        print("=SPLIT(\""+','.join([str(i*100) for i in maj_doc])+"\",\",\")")
        #print("=SPLIT(\""+','.join([str(i*100) for i in classifier_acc])+"\",\",\")")
        print("=SPLIT(\""+','.join([str(i*100) for i in cls_sent])+"\",\",\")")
        print("=SPLIT(\""+','.join([str(i) for i in entorpy])+"\",\",\")")
        print("=SPLIT(\""+','.join([str(i*100) for i in diff_per])+"\",\",\")")
        ##
        print("=SPLIT(\""+','.join([str(i*100) for i in confidence_accuracy])+"\",\",\")")
        print("=SPLIT(\""+','.join([str(i*100) for i in confidence_ratio])+"\",\",\")")
        #gpt
        print("=SPLIT(\""+','.join([str(i) for i in gpt_original])+"\",\",\")")

        print("=SPLIT(\""+','.join([str(i*100) for i in lex_div])+"\",\",\")")
        print("=SPLIT(\""+','.join([str(i) for i in unique_words])+"\",\",\")")


        print("********************")

                
            


print(len(val_step), len(n_acc), len(bt_acc))







FileNotFoundError: [Errno 2] No such file or directory: '../place log path here/stdout'